# Project task 05:  Clustering users based on their preferences

In [56]:
import numpy as np
import scipy.sparse as sp

The goal of this task is to find groups of users with similar preferences using **Spectral clustering**. 
You are given a fragment of the Yelp social network, represented by an undirected weighted graph.
Nodes in the graph represent users.
If two users are connected by an edge of weight $w$, it means that they have both left positive reviews to the same $w$ restaurants.

Additionally, you are given a matrix `F` that encodes user preferences to different categories of restaurants. If `F[i, c] = 1`, then user `i` likes restaurants in category `c`.

You are allowed to use the imported functions (`eigsh`, `KMeans`, `normalize`).

## Load the data

* `N` = number of users (nodes in the graph)
* `C` = number of categories
* The graph is stored as a sparse adjacency matrix `A` (shape `[N, N]`).
* User preferences are stored in a matrix `F` (shape `[N, C]`). They will only be used for the final part of the assignment (Part 3)
* Name of each category is provided in the list `categories` (length `[C]`).

In [57]:
A = sp.load_npz('task_05_data/A.npz')
F = np.load('task_05_data/F.npy',allow_pickle=True)
categories = np.load('task_05_data/categories.npy',allow_pickle=True).tolist()

In [58]:
assert A.shape[0] == F.shape[0]
assert F.shape[1] == len(categories)

# 1. Implementing spectral clustering
## 1.1. Construct the graph Laplacian

First, we need to construct the Laplacian for the given graph. 

Given the **adjacency matrix** $A \in \mathbb{R}^{N \times N},$ we define the **degree matrix** $D \in \mathbb{R}^{N \times N}$ of an undirected graph as
$$D_{ij} =  \begin{cases}\sum_{k=1}^N A_{ik} & if \;\; i = j\\ 0 & if \;\; i \ne j\end{cases}$$

If our goal is to minimize the **ratio cut**, we will need to use the **unnormalized Laplacian**, defined as
$$L_{unnorm} = D - A.$$

If our goal is to minimze the **normalized cut**, we will need to use the **normalized Laplacian** (a.k.a. symmetrized Laplacian), defined as
$$L_{sym} = I - D^{-1/2}AD^{-1/2}$$

In [59]:
def construct_laplacian(A, norm_laplacian):
    """Construct Laplacian of a graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    norm_laplacian : bool
        Whether to construct the normalized graph Laplacian or not.
        If True, construct the normalized (symmetrized) Laplacian, L = I - D^{-1/2} A D^{-1/2}.
        If False, construct the unnormalized Laplacian, L = D - A.
        
    Returns
    -------
    L : scipy.sparse.csr_matrix, shape [N, N]
        Laplacian of the graph.
        
    """
    ### YOUR CODE HERE ###
    
    #assert: diag(A) == 0, should check for this
    I = sp.identity(A.shape[0])
    
    if norm_laplacian:
        #print('debug. shape of sqrt is %s, we were expecting %s'%(str(np.sqrt(np.sum(A,axis = 0)).shape),str(A.shape[0])))
        D_sqrt = sp.diags(np.asarray( 1./np.sqrt(np.sum(A,axis = 0))).squeeze() , 0)
        L = I - D_sqrt.multiply(A.multiply(D_sqrt))
    else:
        #print('debug. shape of sum is %s, we were expecting %s'%(str(np.sum(A,axis = 0).reshape(-1).shape),str(A.shape[0])))
        #print(np.sum(A,axis = 0).toarray().squeeze().shape)
        D = sp.diags(np.asarray(np.sum(A,axis = 0)).squeeze(), 0 )
        L = D - A
    return L

## 1.2. Spectral embedding

Now, we have to compute the spectral embedding for the given graph.

In order to partition the graph into $k$ clusters, such that the desired cut (ratio or normalized) is minimized, we need to consider the $k$ eigenvectors corresponding to the $k$ smallest eigenvalues of the graph Laplacian.

Since the Laplacian matrix is sparse and symmetric, we can use the function `eigsh` from the `scipy.sparse.linalg` package in order to find eigendecomposition of $L$ (`eig` - eigendecomposition, `s` - sparse, `h`- Hermitian).
The function `eigsh` directly allows you to find the smallest / largest eigenvalues by specifying the `k` and `which` parameters. 

Keep in mind that the Laplacian matrix is always positive semi-definite when picking the appropriate value for the `which` parameter.

In [60]:
from scipy.sparse.linalg import eigsh

In [61]:
def spectral_embedding(A, num_clusters, norm_laplacian):
    """Compute spectral embedding of nodes in the given graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    num_clusters : int
        Number of clusters to detect in the data.
    norm_laplacian : bool, default False
        Whether to use the normalized graph Laplacian or not.
        
    Returns
    -------
    embedding : np.array, shape [N, num_clusters]
        Spectral embedding for the given graph.
        Each row represents the spectral embedding of a given node.
    
    """
    if (A != A.T).sum() != 0:
        raise ValueError("Spectral embedding doesn't work if the adjacency matrix is not symmetric.")
    
    L = construct_laplacian(A,norm_laplacian)
    
    # print('debug.printing Laplacian:')
    # print(L.shape)
    # print(L)
    
    _ , eigvecs = sp.linalg.eigsh(L, k=num_clusters, M=None, 
                                         sigma=None, which='SM', v0=None, 
                                         ncv=None, maxiter=None, tol=0, return_eigenvectors=True, Minv=None, OPinv=None, mode='normal')
    
    #according to 1.3, if normlized laplacian was used, we should normalise the 
    # embedding matrix rowwise
    if norm_laplacian is True:
        eigvecs = eigvecs/np.linalg.norm(eigvecs, axis = 1).reshape(-1,1)
        
    # print('debug.printing embedding')
    # print(eigvecs.shape)
    # print(eigvecs)
    
    return eigvecs

## 1.3. Determine the clusters based on the spectral embedding

You should use the K-means algorithm for assigning nodes to clusters, once the spectral embedding is computed.

One thing you should keep in mind, is that when using the **normalized Laplacian**, the rows of the embedding matrix **have to** be normalized to have unit $L_2$ norm.

In [62]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

In [63]:
def spectral_clustering(A, num_clusters, norm_laplacian):
    """Perform spectral clustering on the given graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    num_clusters : int
        Number of clusters to detect in the data.
    norm_laplacian : bool, default False
        Whether to use the normalized graph Laplacian or not.
        
    Returns
    -------
    z_pred : np.array, shape [N]
        Predicted cluster indicators for each node.
        
    """
    
    embedding  = spectral_embedding(A, num_clusters, norm_laplacian)
    kmeans = KMeans(n_clusters=num_clusters, init='k-means++', n_init=10, 
                    max_iter=300, tol=0.0001, precompute_distances='auto', verbose=0,
                    random_state=None, copy_x=True, n_jobs=None, algorithm='auto').fit(embedding[:,1:])
    z_pred = kmeans.labels_
    
    return z_pred

# 2. Quantitatively evaluate the results

In [64]:
def labels_to_list_of_clusters(z):
    """Convert predicted label vector to a list of clusters in the graph.
    This function is already implemented, nothing to do here.
    
    Parameters
    ----------
    z : np.array, shape [N]
        Predicted labels.
        
    Returns
    -------
    list_of_clusters : list of lists
        Each list contains ids of nodes that belong to the same cluster.
        Each node may appear in one and only one partition.
    
    Examples
    --------
    >>> z = np.array([0, 0, 1, 1, 0])
    >>> labels_to_list_of_clusters(z)
    [[0, 1, 4], [2, 3]]
    
    """
    return [np.where(z == c)[0] for c in np.unique(z)]

## 2.1. Compute ratio cut

Your task is to implement functions for computing the **ratio cut** and **normalized cut** for a given partition.

Ratio cut and normalized cut are defined on the slide 14 of the lecture slides.


The function `labels_to_list_of_clusters` can be helpful here.

In [65]:
def compute_ratio_cut(A, z):
    """Compute the ratio cut for the given partition of the graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    z : np.array, shape [N]
        Cluster indicators for each node.
    
    Returns
    -------
    ratio_cut : float
        Value of the cut for the given partition of the graph.
        
    """
    
    ### YOUR CODE HERE ###
    # print('z is %s'%str(z))
    
    sum = 0.
    for category in range(z.max()):
        indices = np.argwhere(z == category).reshape(-1)
        mask = np.ones(A.shape[0])
        mask[indices] = 0.
        
        #print('debug.shapes of indices: %s . Indices is %s'%(str(indices.shape),str(indices)))
        #print('debug. shapes of A[indices] and mask : %s , %s'%(str(A[indices].shape),str(mask.shape)))
        cut = np.sum (A[indices]*mask)
        cut = cut / indices.shape[0]
        sum  = sum+cut
        
    return sum

## 2.2. Compute normalized cut

**Important**: if a cluster only contains a single node, define its volume to be 1 to avoid division by zero errors.

In [66]:
def compute_normalized_cut(A, z):
    """Compute the normalized cut for the given partition of the graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    z : np.array, shape [N]
        Cluster indicators for each node.
    
    Returns
    -------
    norm_cut : float
        Value of the normalized cut for the given partition of the graph.
        
    """
    
    sum = 0.
    for category in range(z.max()):
        indices = np.argwhere(z == category).reshape(-1)
        mask = np.ones(A.shape[0])
        mask[indices] = 0.
        cut = np.sum (A[indices]*mask)
        
        #compute volume of category:
        volume = np.sum(A[indices])
        if volume is not 0.:
            cut = cut/volume
        sum  = sum+cut
        
    return sum

Notice, how using the unnormalized Laplacian leads to a much better ratio cut, while the normalized Laplacian leads to better normalized cut.

In [67]:
num_clusters = 6

In [68]:
norm_laplacian = False
z_unnorm = spectral_clustering(A, num_clusters, norm_laplacian)
print('When using L_unnorm:')
print(' ratio cut = {:.3f}'.format(compute_ratio_cut(A, z_unnorm)))
print(' normalized cut = {:.3f}'.format(compute_normalized_cut(A, z_unnorm)))
print(' sizes of partitions are: {}'.format([len(clust) for clust in labels_to_list_of_clusters(z_unnorm)]))

When using L_unnorm:
 ratio cut = 287.109
 normalized cut = 4.000
 sizes of partitions are: [3379, 1, 1, 1, 1, 1]


In [69]:
norm_laplacian = True
z_norm = spectral_clustering(A, num_clusters, norm_laplacian)
print('When using L_norm:')
print(' ratio cut = {:.3f}'.format(compute_ratio_cut(A, z_norm)))
print(' normalized cut = {:.3f}'.format(compute_normalized_cut(A, z_norm)))
print(' sizes of partitions are: {}'.format([len(clust) for clust in labels_to_list_of_clusters(z_norm)]))

When using L_norm:
 ratio cut = 5347.316
 normalized cut = 4.198
 sizes of partitions are: [554, 586, 491, 507, 607, 639]


# 3. Visualize the results

In the final part of the assignment, your task is to print out the 5 most popular types of restaurants visited by the users in each cluster.

In [87]:
def print_top_categories_for_each_cluster(top_k, z, F, categories):
    """Print the top-K categories among users in each cluster.
    For each cluster, the function prints names of the top-K categories,
    and number of users that like the respective category (separated by a comma).
    The function doesn't return anything, just prints the output.
    
    Parameters
    ----------
    top_k : int
        Number of most popular categories to print for each cluster.
    z : np.array, shape [N]
        Cluster labels.
    F : sp.csr_matrix, shape [N, C]
        Matrix that tells preferences of each user to each category.
        F[i, c] = 1 if user i gave at least one positive review to at least one restaurant in category c.
    categories : list, shape [C]
        Names of the categories.
        
    """
    for cluster in range(z.max()):
        cluster_votes  = F[np.argwhere(cluster == z).reshape(-1)]
        #print(cluster_votes)
        cluster_total_votes = np.sum(cluster_votes,axis = 0).reshape(-1)
        top_categories_idx = np.argsort(cluster_total_votes)[-top_k:]
        #print(top_categories_idx)
        print('')
        print('Most popular restaurant categories in user group %d:'%int(cluster))
        for idx in top_categories_idx:
            print('%s. Number of voters: %d'%(categories[idx],cluster_total_votes[idx]))
            
        
            

In [89]:
np.random.seed(2)
z_norm = spectral_clustering(A, num_clusters, True)
r = print_top_categories_for_each_cluster(5, z_norm, F, categories)


Most popular restaurant categories in user group 0:
Asian Fusion. Number of voters: 399
Sandwiches. Number of voters: 433
Italian. Number of voters: 449
Japanese. Number of voters: 463
Breakfast & Brunch. Number of voters: 525

Most popular restaurant categories in user group 1:
Pizza. Number of voters: 329
Sandwiches. Number of voters: 395
Italian. Number of voters: 398
Japanese. Number of voters: 421
Breakfast & Brunch. Number of voters: 441

Most popular restaurant categories in user group 2:
Asian Fusion. Number of voters: 411
Sandwiches. Number of voters: 459
Italian. Number of voters: 490
Japanese. Number of voters: 499
Breakfast & Brunch. Number of voters: 541

Most popular restaurant categories in user group 3:
Pizza. Number of voters: 307
Italian. Number of voters: 349
Sandwiches. Number of voters: 353
Japanese. Number of voters: 376
Breakfast & Brunch. Number of voters: 415

Most popular restaurant categories in user group 4:
Cafes. Number of voters: 348
Sandwiches. Number o